### Basic Blockchain in Python
In this assignment we will seek to reproduce some of the Blockchain mechanisms found in Bitcoin in a short series of classes written in Python.

In [1]:
#import necessary packages

import hashlib
import json
import time


In [2]:
#Details of transactions

tx_1 = {
   "addr_from": "3J98t1WpEZ73CNmQviecrnyiWrnqRhWNLy",
   "addr_to": "1BvBMSEYstWetqTFn5Au4m4GFg7xJaNVN2",   
   "amount": 99.00
}

tx_2 = {
   "addr_from": "1BvBMSEYstWetqTFn5Au4m4GFg7xJaNVN2",
   "addr_to": "3J98t1WpEZ73CNmQviecrnyiWrnqRhWNLy",   
   "amount": 99.00
}
tx_3 = {
   "addr_from": "FintechStudentAddr",
   "addr_to": "BlackBelts",   
   "amount": 9999.99
}

In [3]:
#Define Block class
#And Define it's constructor and properties.

class Block():
    def __init__(self,
                 index:int = 0,
                 transactions:list = [],
                 timestamp:int = 0,
                 cur_hash:str = "",
                 previous_hash:str = "",
                 nonce:int = 0):
        
        self.index:int = index
        self.transactions:list = transactions
        self.timestamp = timestamp
        self.hash:str = cur_hash
        self.previous_hash:str = previous_hash
        self.nonce:int = nonce
            
    def compute_hash(self) -> str:
        block_string = json.dumps(self.__dict__, sort_keys=True)
        return hashlib.sha256(block_string.encode()).hexdigest()

In [4]:
#Define Blockchain class
#And Define it's constructor and properties.

class Blockchain():     
    def __init__(self):
        self.unconfirmed_transactions:List[dict] = []  #mem-pool
        self.chain:List[Block] = []
        self.difficulty:int = 2
        self.create_genesis_block()
   
    def get_last_block(self) -> Block:
        return self.chain[-1]
    
    def create_genesis_block(self):
        genesis_block = Block(index=0, transactions=[], timestamp=time.time(), previous_hash="0")  
        genesis_block.hash = genesis_block.compute_hash()
        self.chain.append(genesis_block)
        
    def proof_of_work(self, block:Block) -> str:
        computed_hash = block.compute_hash()
        while not computed_hash.startswith('0' * self.difficulty):
            block.nonce += 1
            computed_hash = block.compute_hash()
        return computed_hash
        
    def is_valid_proof(self, block:Block, block_hash:str) -> bool:
        return block_hash.startswith('0' * self.difficulty) and block_hash == block.compute_hash()
   
    def add_block(self, block:Block, proof:str) -> bool:
        last_block = self.get_last_block()
        last_block.hash
        
        if last_block.hash != block.previous_hash:
            return False
        
        if not self.is_valid_proof(block , proof):
            return False 
    
        block.hash = proof
        self.chain.append(block)
        
        return True
        
        
    def add_new_transaction(self, transaction:dict) -> None:
        self.unconfirmed_transactions.append(transaction)
        

    def mine(self) -> int:
        if len(self.unconfirmed_transactions)<1:
            return -1
        
        last_block = self.get_last_block()
#        self.unconfirmed_transactions.append(tx_1)#just for testing purposes.
        
        new_block = Block(index=last_block.index + 1,
                         transactions=self.unconfirmed_transactions,
                         timestamp=int(time.time()),
                         previous_hash=last_block.hash)
 
        proof = self.proof_of_work(new_block)
    
        self.add_block(new_block ,proof)
        
        self.unconfirmed_transactions = []
        
        return new_block.index



    
    

In [5]:
#Generate some transactions

bc = Blockchain()
[bc.add_new_transaction(tx) for tx in [tx_1,tx_2,tx_3]]

[None, None, None]

In [6]:
bc.unconfirmed_transactions

[{'addr_from': '3J98t1WpEZ73CNmQviecrnyiWrnqRhWNLy',
  'addr_to': '1BvBMSEYstWetqTFn5Au4m4GFg7xJaNVN2',
  'amount': 99.0},
 {'addr_from': '1BvBMSEYstWetqTFn5Au4m4GFg7xJaNVN2',
  'addr_to': '3J98t1WpEZ73CNmQviecrnyiWrnqRhWNLy',
  'amount': 99.0},
 {'addr_from': 'FintechStudentAddr',
  'addr_to': 'BlackBelts',
  'amount': 9999.99}]

In [7]:
bc.chain[0].__dict__

{'index': 0,
 'transactions': [],
 'timestamp': 1639062079.525516,
 'hash': 'a1416873b6f61a1eb48a1efe185e32b8850fcec43152aa4e67b743f5d6a553e9',
 'previous_hash': '0',
 'nonce': 0}

In [8]:
bc.get_last_block().__dict__

{'index': 0,
 'transactions': [],
 'timestamp': 1639062079.525516,
 'hash': 'a1416873b6f61a1eb48a1efe185e32b8850fcec43152aa4e67b743f5d6a553e9',
 'previous_hash': '0',
 'nonce': 0}

In [9]:
bc.mine()

1

In [10]:
bc.get_last_block().__dict__

{'index': 1,
 'transactions': [{'addr_from': '3J98t1WpEZ73CNmQviecrnyiWrnqRhWNLy',
   'addr_to': '1BvBMSEYstWetqTFn5Au4m4GFg7xJaNVN2',
   'amount': 99.0},
  {'addr_from': '1BvBMSEYstWetqTFn5Au4m4GFg7xJaNVN2',
   'addr_to': '3J98t1WpEZ73CNmQviecrnyiWrnqRhWNLy',
   'amount': 99.0},
  {'addr_from': 'FintechStudentAddr',
   'addr_to': 'BlackBelts',
   'amount': 9999.99}],
 'timestamp': 1639062079,
 'hash': '007c8224c890758c547b27988c973bb6ec2a0982bb2d97bf7236df2ebb852374',
 'previous_hash': 'a1416873b6f61a1eb48a1efe185e32b8850fcec43152aa4e67b743f5d6a553e9',
 'nonce': 210}

In [11]:
[bc.add_new_transaction(tx) for tx in [{"FintechData1":"SomeInsuranceStuff"},{"FintechData2":"SomeCapitalMarketStuff"},{"FintechData3":"SomePaymentsStuff"}]]
 
 

[None, None, None]

In [12]:
bc.unconfirmed_transactions

[{'FintechData1': 'SomeInsuranceStuff'},
 {'FintechData2': 'SomeCapitalMarketStuff'},
 {'FintechData3': 'SomePaymentsStuff'}]

In [13]:
bc.mine()

2

In [14]:
bc.get_last_block().__dict__

{'index': 2,
 'transactions': [{'FintechData1': 'SomeInsuranceStuff'},
  {'FintechData2': 'SomeCapitalMarketStuff'},
  {'FintechData3': 'SomePaymentsStuff'}],
 'timestamp': 1639062079,
 'hash': '00448a02dc9ba23a4d8aa6ed63b13b21210cb70d9498f476d83db89ee96b9d5e',
 'previous_hash': '007c8224c890758c547b27988c973bb6ec2a0982bb2d97bf7236df2ebb852374',
 'nonce': 858}

In [15]:
b = Block()
b.compute_hash()

'0267061eb16c636898bb6afeefc4959a18d5210d0415e315d29f2db21d048782'